In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
file_list = os.listdir("/works/Data/Dacon/health_data_clf/")
file_list = [file for file in file_list if file.endswith(".csv")]
print(file_list)

['train_labels.csv', 'test_features.csv', 'train_features.csv', 'sample_submission.csv']


In [3]:
file_path = "/works/Data/Dacon/health_data_clf/"
train_label = pd.read_csv(file_path + file_list[0])
test_features = pd.read_csv(file_path + file_list[1])
train_features = pd.read_csv(file_path + file_list[2])
sample_submissuib = pd.read_csv(file_path + file_list[3])

print(train_label.shape)
print(test_features.shape)
print(train_features.shape)

(3125, 3)
(469200, 8)
(1875000, 8)


# Preprocessing

In [4]:
def return_index(arr, num):
    return arr.argsort()[-num:]

def prerpocessing_using_static(features_set, label_set):
    input_set = []
    target_set = []
    
    for id_idx in range(features_set.id.min(), features_set.id.max()+1):
        sample_features = features_set[["acc_x","acc_y","acc_z", "gy_x", "gy_y", "gy_z"]][features_set.id==id_idx].values
        
        #Fourier transform
        strength = np.fft.fft(sample_features, axis=0)
        strength = abs(strength)
        frequency = np.fft.fftfreq(len(sample_features), 1)
        strength_pos = strength[frequency>=0]
        frequency_pos = frequency[frequency>=0]
        freq_x = frequency_pos[return_index(strength_pos[:,0],3)].mean()
        freq_y = frequency_pos[return_index(strength_pos[:,1],3)].mean()
        freq_z = frequency_pos[return_index(strength_pos[:,2],3)].mean()
        freq_gy_x = frequency_pos[return_index(strength_pos[:,3],3)].mean()
        freq_gy_y = frequency_pos[return_index(strength_pos[:,4],3)].mean()
        freq_gy_z = frequency_pos[return_index(strength_pos[:,5],3)].mean()
        
        #Transform to static
        #Mean values
        mean_arr = sample_features.mean(axis=0)
        #Mag values
        mag_arr = abs(sample_features).mean(axis=0)
        #Std values
        std_arr = sample_features.std(axis=0)
        #Cov values
        cov_xy = np.cov(sample_features[:,0], sample_features[:,1])[0][1]
        cov_yz = np.cov(sample_features[:,1], sample_features[:,2])[0][1]
        cov_zx = np.cov(sample_features[:,2], sample_features[:,0])[0][1]
        cov_gy_xy = np.cov(sample_features[:,3], sample_features[:,4])[0][1]
        cov_gy_yz = np.cov(sample_features[:,4], sample_features[:,5])[0][1]
        cov_gy_zx = np.cov(sample_features[:,5], sample_features[:,3])[0][1]
        #Corr values
        cor_xy = cov_xy / (std_arr[0]*std_arr[1])
        cor_yz = cov_yz / (std_arr[1]*std_arr[2])
        cor_zx = cov_zx / (std_arr[2]*std_arr[0])
        cor_gy_xy = cov_gy_xy / (std_arr[3]*std_arr[4])
        cor_gy_yz = cov_gy_yz / (std_arr[4]*std_arr[5])
        cor_gy_zx = cov_gy_zx / (std_arr[5]*std_arr[3])
        #Start End point change of vector values
        start_point = sample_features[:3,:3].mean(axis=0)
        end_point = sample_features[-3:,:3].mean(axis=0)
        change_of_vector_st = np.cos((start_point * end_point).sum() / (np.sqrt(np.power(start_point, 2).sum()) * np.sqrt(np.power(end_point, 2).sum())))
        start_gy_point = sample_features[:3,3:].mean(axis=0)
        end_gy_point = sample_features[-3:,3:].mean(axis=0)
        change_of_vector_st_gy = np.cos((start_gy_point * end_gy_point).sum() / (np.sqrt(np.power(start_gy_point, 2).sum()) * np.sqrt(np.power(end_gy_point, 2).sum())))
        
        #Energy values
        shift_0_values = np.roll(sample_features, 1, axis=0)[1:]
        shift_1_values = sample_features[1:]
        energys = np.power((shift_0_values-shift_1_values),2)
        energy_strength = np.fft.fft(energys, axis=0)
        energy_strength = abs(energy_strength)
        energy_frequency = np.fft.fftfreq(len(energy_strength), 1)
        energy_strength_pos = energy_strength[energy_frequency>=0]
        energy_frequency_pos = energy_frequency[energy_frequency>=0]
        energy_freq_x = energy_frequency_pos[return_index(energy_strength_pos[:,0],3)].mean()
        energy_freq_y = energy_frequency_pos[return_index(energy_strength_pos[:,1],3)].mean()
        energy_freq_z = energy_frequency_pos[return_index(energy_strength_pos[:,2],3)].mean()
        energy_freq_gy_x = energy_frequency_pos[return_index(energy_strength_pos[:,3],3)].mean()
        energy_freq_gy_y = energy_frequency_pos[return_index(energy_strength_pos[:,4],3)].mean()
        energy_freq_gy_z = energy_frequency_pos[return_index(energy_strength_pos[:,5],3)].mean()
        energy_mean = energys.mean(axis=0)
        energy_std = energys.std(axis=0)
        #Max Min point num
        increase_shift_0 = np.where(shift_0_values-shift_1_values>=0, 1, -1)
        increase_shift_1 = np.roll(increase_shift_0, 1, axis=0)
        max_min_point_num = np.where(increase_shift_0[1:]*increase_shift_1[1:]==-1, 1, 0).sum(axis=0)
        
        #Make Set
        id_sample_set = [mean_arr[0], mag_arr[0]] + list(mag_arr[3:]) + list(std_arr)\
        + [energy_mean[0], energy_mean[1], energy_std[0]]\
        + list(max_min_point_num) + [freq_gy_x, freq_gy_y, freq_gy_z]
        
        #Append
        input_set.append(id_sample_set)
        target_set.append(label_set[label_set.id == id_idx].label.values[0])
    
    input_set = np.array(input_set)
    target_set = np.array(target_set)
    print("Input set : {} / Target set : {}".format(input_set.shape, target_set.shape))
    return input_set, target_set

In [5]:
input_set, target_set = prerpocessing_using_static(features_set=train_features, label_set=train_label)

Input set : (3125, 23) / Target set : (3125,)


# Data Split

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(input_set, target_set, test_size=0.1, stratify=target_set)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(2812, 23) (2812,) (313, 23) (313,)


# Model Part

In [7]:
import lightgbm as lgb

In [18]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)
params = {
    'learning_rate' : 0.05,
    'boosting_type' : 'dart',
    'max_depth' : 3,
    'num_leaves' : 2,
    'min_data_in_leaf': 100,
    'drop_rate' : 0.6,
    'feature_fraction' : 0.2,
    'bagging_fraction' : 0.2,
    'objective' : 'multiclass',
    'metric' : 'multi_logloss',
    'num_class':61
}

clf = lgb.train(params, train_ds, 1300, val_ds, verbose_eval=100, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000225 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5389
[LightGBM] [Info] Number of data points in the train set: 2812, number of used features: 23
[LightGBM] [Info] Start training from score -5.543756
[LightGBM] [Info] Start training from score -4.997212
[LightGBM] [Info] Start training from score -5.051279
[LightGBM] [Info] Start training from score -4.897129
[LightGBM] [Info] Start training from score -4.507664
[LightGBM] [Info] Start training from score -4.806157
[LightGBM] [Info] Start training from score -4.850609
[LightGBM] [Info] Start training from score -4.806157
[LightGBM] [Info] Start training from score -3.475743
[LightGBM] [Info] Start training from score -4.445144
[LightGBM] [Info] Start training from score -5.051279
[LightGBM] [Info] Start training from score -4.897129
[LightGBM] [Info] Start training from score -5.543756
[LightG

# Result

In [19]:
train_pred = clf.predict(x_train)
val_pred = clf.predict(x_val)

def return_armax(arrs):
    return np.argmax(arrs, axis=1)
train_pred = return_armax(train_pred)
val_pred = return_armax(val_pred)

from sklearn.metrics import confusion_matrix, accuracy_score

print(accuracy_score(y_train, train_pred))
print(accuracy_score(y_val, val_pred))

0.9591038406827881
0.7667731629392971


# For Submit

In [20]:
def return_index(arr, num):
    return arr.argsort()[-num:]

def prerpocessing_using_static_sub(features_set):
    input_set = []

    for id_idx in range(features_set.id.min(), features_set.id.max()+1):
        sample_features = features_set[["acc_x","acc_y","acc_z", "gy_x", "gy_y", "gy_z"]][features_set.id==id_idx].values
        
        #Fourier transform
        strength = np.fft.fft(sample_features, axis=0)
        strength = abs(strength)
        frequency = np.fft.fftfreq(len(sample_features), 1)
        strength_pos = strength[frequency>=0]
        frequency_pos = frequency[frequency>=0]
        freq_x = frequency_pos[return_index(strength_pos[:,0],3)].mean()
        freq_y = frequency_pos[return_index(strength_pos[:,1],3)].mean()
        freq_z = frequency_pos[return_index(strength_pos[:,2],3)].mean()
        freq_gy_x = frequency_pos[return_index(strength_pos[:,3],3)].mean()
        freq_gy_y = frequency_pos[return_index(strength_pos[:,4],3)].mean()
        freq_gy_z = frequency_pos[return_index(strength_pos[:,5],3)].mean()
        
        #Transform to static
        #Mean values
        mean_arr = sample_features.mean(axis=0)
        #Mag values
        mag_arr = abs(sample_features).mean(axis=0)
        #Std values
        std_arr = sample_features.std(axis=0)
        #Cov values
        cov_xy = np.cov(sample_features[:,0], sample_features[:,1])[0][1]
        cov_yz = np.cov(sample_features[:,1], sample_features[:,2])[0][1]
        cov_zx = np.cov(sample_features[:,2], sample_features[:,0])[0][1]
        cov_gy_xy = np.cov(sample_features[:,3], sample_features[:,4])[0][1]
        cov_gy_yz = np.cov(sample_features[:,4], sample_features[:,5])[0][1]
        cov_gy_zx = np.cov(sample_features[:,5], sample_features[:,3])[0][1]
        #Corr values
        cor_xy = cov_xy / (std_arr[0]*std_arr[1])
        cor_yz = cov_yz / (std_arr[1]*std_arr[2])
        cor_zx = cov_zx / (std_arr[2]*std_arr[0])
        cor_gy_xy = cov_gy_xy / (std_arr[3]*std_arr[4])
        cor_gy_yz = cov_gy_yz / (std_arr[4]*std_arr[5])
        cor_gy_zx = cov_gy_zx / (std_arr[5]*std_arr[3])
        #Start End point change of vector values
        start_point = sample_features[:3,:3].mean(axis=0)
        end_point = sample_features[-3:,:3].mean(axis=0)
        change_of_vector_st = np.cos((start_point * end_point).sum() / (np.sqrt(np.power(start_point, 2).sum()) * np.sqrt(np.power(end_point, 2).sum())))
        start_gy_point = sample_features[:3,3:].mean(axis=0)
        end_gy_point = sample_features[-3:,3:].mean(axis=0)
        change_of_vector_st_gy = np.cos((start_gy_point * end_gy_point).sum() / (np.sqrt(np.power(start_gy_point, 2).sum()) * np.sqrt(np.power(end_gy_point, 2).sum())))
        
        #Energy values
        shift_0_values = np.roll(sample_features, 1, axis=0)[1:]
        shift_1_values = sample_features[1:]
        energys = np.power((shift_0_values-shift_1_values),2)
        energy_strength = np.fft.fft(energys, axis=0)
        energy_strength = abs(energy_strength)
        energy_frequency = np.fft.fftfreq(len(energy_strength), 1)
        energy_strength_pos = energy_strength[energy_frequency>=0]
        energy_frequency_pos = energy_frequency[energy_frequency>=0]
        energy_freq_x = energy_frequency_pos[return_index(energy_strength_pos[:,0],3)].mean()
        energy_freq_y = energy_frequency_pos[return_index(energy_strength_pos[:,1],3)].mean()
        energy_freq_z = energy_frequency_pos[return_index(energy_strength_pos[:,2],3)].mean()
        energy_freq_gy_x = energy_frequency_pos[return_index(energy_strength_pos[:,3],3)].mean()
        energy_freq_gy_y = energy_frequency_pos[return_index(energy_strength_pos[:,4],3)].mean()
        energy_freq_gy_z = energy_frequency_pos[return_index(energy_strength_pos[:,5],3)].mean()
        energy_mean = energys.mean(axis=0)
        energy_std = energys.std(axis=0)
        #Max Min point num
        increase_shift_0 = np.where(shift_0_values-shift_1_values>=0, 1, -1)
        increase_shift_1 = np.roll(increase_shift_0, 1, axis=0)
        max_min_point_num = np.where(increase_shift_0[1:]*increase_shift_1[1:]==-1, 1, 0).sum(axis=0)
        
        #Make Set
        id_sample_set = [mean_arr[0], mag_arr[0]] + list(mag_arr[3:]) + list(std_arr)\
        + [energy_mean[0], energy_mean[1], energy_std[0]]\
        + list(max_min_point_num) + [freq_gy_x, freq_gy_y, freq_gy_z]
        
        #Append
        input_set.append(id_sample_set)
    
    input_set = np.array(input_set)
    print("Input set : {}".format(input_set.shape))
    return input_set

In [21]:
sub_input_set =  prerpocessing_using_static_sub(features_set=test_features)
print(sub_input_set.shape)

Input set : (782, 23)
(782, 23)


In [22]:
sub_pred = clf.predict(sub_input_set)

In [23]:
result_df = pd.DataFrame()
result_df['id'] = sample_submissuib.id
for i in range(61):
    result_df[i] = sub_pred[:,i]

In [24]:
result_df.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000885,0.000019,2.030462e-04,0.001398,8.685772e-04,0.000002,1.078648e-03,0.000007,0.000395,...,0.098816,0.006190,0.002231,0.000192,2.589344e-04,0.000009,0.000103,0.064930,1.669344e-07,0.002858
1,3126,0.000176,0.000019,1.455337e-05,0.000036,1.547504e-08,0.000113,7.468432e-06,0.000037,0.000015,...,0.000018,0.000012,0.000076,0.000012,1.748706e-05,0.000021,0.000033,0.000044,3.445375e-08,0.000079
2,3127,0.000062,0.037432,2.298600e-04,0.000699,1.974037e-07,0.000214,1.788182e-04,0.000191,0.000689,...,0.002554,0.000021,0.000136,0.641515,3.792500e-04,0.000491,0.000009,0.000329,2.465759e-04,0.000519
3,3128,0.000901,0.000042,7.537984e-06,0.000001,2.024373e-08,0.000007,5.169889e-07,0.000052,0.000020,...,0.000201,0.000016,0.000009,0.000256,2.074373e-04,0.000115,0.000011,0.000003,3.373644e-08,0.004036
4,3129,0.009225,0.001996,8.527394e-07,0.000166,7.772278e-05,0.000016,6.012295e-05,0.002719,0.000167,...,0.000090,0.000068,0.000010,0.000002,7.217129e-07,0.000018,0.000479,0.000219,1.484582e-07,0.000633


In [25]:
result_df.to_csv("/works/Data/Dacon/health_data_clf/result/210217_2.csv",index=False)